In [109]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
from random import randrange
import math

In [110]:
data = pd.read_csv('agaricus-lepiota.data')

In [173]:
def hamming_distance(row, cluster_modes):
    distances = np.array([])
    for cluster_mode in cluster_modes:
        distances = np.append(distances, row.eq(cluster_mode).value_counts().get(False, 0))
    print(distances)
    print(np.argmin(distances))

In [174]:
def k_mode_clustering(data, number_of_clusters):
    cluster_centroids = []
    clusters = {}
    for i in range(number_of_clusters):
        rand_index = randrange(len(data))
        cluster_centroids.append(data.loc[rand_index])
    data.apply(hamming_distance, args=[cluster_centroids], axis=1)
    

In [ ]:
k_mode_clustering(data.head(20),4)